# Atividade 2 - Visão Computacional

O entregável de toda esta atividade vai ser um código-fonte em *Python*. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube



Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera

Detalhe como calculou $f$

## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

import auxiliar as aux
from ipywidgets import widgets, interact, interactive, FloatSlider, IntSlider
from scipy.spatial.distance import euclidean as dist

# If you want to open a video, just change this path
#cap = cv2.VideoCapture('hall_box_battery.mp4')

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1
magenta = '#de3163'
ciano = '#4682B4'
m1, m2 = aux.ranges(magenta)
c1, c2 = aux.ranges(ciano)
# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged



while(True):
    # Capture frame-by-frame
    #print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask_magenta = cv2.inRange(frame_hsv, m1, m2)
    mask_ciano = cv2.inRange(frame_hsv, c1, c2)
    cor = (0, 255, 0)

    circles = []


    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    centro= []
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            mask_magenta = cv2.inRange(frame_hsv, m1, m2)
            mask_ciano = cv2.inRange(frame_hsv, c1, c2)
            #print(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(int(frame[i[1],i[0]][0]),int(frame[i[1],i[0]][1]),int(frame[i[1],i[0]][2])),7)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(int(frame[i[1],i[0]][0]),int(frame[i[1],i[0]][1]),int(frame[i[1],i[0]][2])),3)
            centro.append((i[0],i[1]))
        if len(centro)>= 2:
            cv2.line(bordas_color, centro[0], centro[1], (255,255,255), 2)
            length = np.sqrt(int((centro[0][0])- int(centro[1][0]))**2 + (int(centro[0][1])- int(centro[1][1])) ** 2)
            dist = 6650/length
            print(dist)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(bordas_color,str(dist),(0,50), font, 2,(255,255,255),2,cv2.LINE_AA)

    #More drawing functions @ http://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html

    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    #print("No circles were found")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

40.15653625169825
40.22406328413627
40.776832118246254
145.53129641952583
39.17462046268395
36.88756304897774
33.328414017928715
41.48156029319505
41.162466629246495
40.630437633583895
24.810669493439608
23.918387542081412
23.566747558270183
23.566747558270183
22.30739934899277
22.310914420889677
22.017936321148266
22.017936321148266
21.874526612493046
21.59090909090909
21.59090909090909
21.45116647360539
21.31366466457682
21.31366466457682
21.447596039746838
21.579538152417825
21.307099408807854
21.312351127921943
21.312351127921943
21.312351127921943
21.17662576439556
21.310162439738722
21.174478621542303
21.17662576439556
21.042618027946084
21.042618027946084
20.7747589019188
21.040511385632204
21.17662576439556
21.174478621542303
21.16761215079133
21.16761215079133
20.910295520867745
20.908228366938435
20.270998963950174
20.524300333317658
20.151145069604585
20.151145069604585
19.33139534883721
18.57281071814169
18.2648200422787
18.262340411830195
18.262340411830195
18.356443184900

## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

## Parte 5

Usando transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

## Parte 6

Usando `SIFT`, identifique o escrito *Insper* na folha